# ProPyCore SDK Examples: Generic Tools

This notebook contains snippets from the `generic_tools` module which helps to handle correspondence data

In [8]:
import os
import dotenv
import json
import ProPyCore

from datetime import datetime

## Setup
You will need to create the connection to your Procore app and then get the details for your company and project.

### Connection to Procore App
Ensure you have a `.env` file with the following information included:
* `CLIENT_ID`: your data connection app's client ID
* `CLIENT_SECRET`: your data connection app's client secret

In [9]:
dotenv.load_dotenv()
connection = ProPyCore.procore.Procore(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    redirect_uri="urn:ietf:wg:oauth:2.0:oob", # default for data connection apps
    oauth_url="https://app.procore.com", # default for data connection apps
    base_url="https://app.procore.com" # default for data connection apps
)

### Get Company Details
Use the cell below to specify your company name

In [10]:
companies = connection.companies.get()
for company in companies:
    print(company["name"])

company_name = companies[0]["name"]

Rogers-O`Brien Construction


Now get the company details

In [11]:
company = connection.companies.find(identifier=company_name)
projects = connection.projects.get(company_id=company["id"])
project = connection.projects.find(
    company_id=company["id"],
    identifier="Sandbox Test Project"
)
print(project["name"])

Sandbox Test Project


### `tools.get()`

```python
"""
Gets all the available generic tools

Parameters
----------
company_id : int
    unique identifier for the company
per_page : int, default 100
    number of generic tools to find per page

Returns
-------
tools : dict
    available generic tools
"""
```

In [12]:
tools = connection.tools.get(
    company_id=company["id"]
)
for tool in tools:
    print(f"{tool['id']}: {tool['title']}")

try:
    idea_submission_id = 378532
    idea_submission_tool = next(item for item in tools if item["id"] == idea_submission_id)
    print(json.dumps(idea_submission_tool, indent=4))
except StopIteration:
    print("No such tool with id", idea_submission_id)

AttributeError: 'GenericTool' object has no attribute 'get'

### `tools.find()`

```python
"""
Finds a tools based on the identifier: id or title

Parameters
----------
company_id : int
    company id that the project is under
identifier : int or str
    project id number or company name

Returns
-------
tool : dict
    tool-specific dictionary
"""
```

In [6]:
project = connection.projects.find(
    company_id=company["id"],
    identifier="Sandbox Test Project"
)
tool = connection.tools.find(
    company_id=company["id"],
    identifier="Idea Submission"
)
print("Company:", company["id"])
print("Project:", project["id"])
print("Tool:", tool["id"])

Number of Timecards: 1


### `tools.get_items()`
```python
"""
Gets all the available items for a specific tool

Parameters
----------
company_id : int
    unique identifier for the company
project_id : int
    unique identifier for the project
tool_id : int
    unique identifier for the generic tool

Returns
-------
items : dict
    available tool item data
"""
```

In [7]:
project = connection.projects.find(
    company_id=company["id"],
    identifier="Sandbox Test Project"
)
tool = connection.tools.find(
    company_id=company["id"],
    identifier="Idea Submission"
)

# list all items for idea submission tool
print("Example 1")
tool_items = connection.tools.get_items(
    company_id=company["id"],
    project_id=project["id"],
    tool_id=tool["id"]
)
print(tool_items)

Number of Timecards: 4


### `tools.get_statuses()`
```python
"""
Gets all the available statuses for a specific tool

Parameters
----------
company_id : int
    unique identifier for the company
tool_id : int
    unique identifier for the generic tool

Returns
-------
items : dict
    available tool item data
"""
```

In [9]:
project = connection.projects.find(
    company_id=company["id"],
    identifier="Sandbox Test Project"
)
tool = connection.tools.find(
    company_id=company["id"],
    identifier="Idea Submission"
)

# list all statuses for idea submission tool
tool_statuses = connection.tools.get_statuses(
    company_id=company["id"],
    project_id=project["id"],
    tool_id=tool["id"]
)
print(tool_statuses)

{
    "timecard_entry": {
        "hours": "8",
        "party_id": 25138567,
        "date": "2025-02-06",
        "cost_code_id": 971232406,
        "billable": true,
        "description": "Test timecard from API",
        "timecard_time_type_id": 89458,
        "line_item_type_id": 832349,
        "datetime": "2025-02-06T12:00:00Z"
    }
}


### `tools.create_item()`
```python
"""
Create new item for a specific tool

Parameters
----------
company_id : int
    unique identifier for the company
project_id : int
    unique identifier for the project
tool_id : int
    unique identifier for the generic tool
data : dict
    request body data for the new item

Returns
-------
item_info : dict
    new item data
"""
```

In [ ]:
# create new idea submission
data = {
    "generic_tool_item": {
        "custom_field_378362": "This idea was submitted via API", # this field is unique
        "description": "None",
        "private": False,
        "received_from_id": 8780450, # this field is unique to your company
        "status":"Open",
        "title": "Simple Idea Submited by API",
    }
}
# create the item
tool_item = connection.tools.create_item(
    company_id=company["id"],
    project_id=project["id"],
    tool_id=tool["id"],
    data=data
)
# show created item
print(tool_item)

In [ ]:

# create new idea submission
data = {
    "generic_tool_item": {
        "assignee_ids": [
            8780450
        ],
        "attachments": [
            {
            "id": random.randint(1, 100),
            "url": "https://innovate.r-o.com",
            "filename": f"{PATH_TO_TOP}/references/sample_correspondence_file.pdf"
            }
        ],
        "cost_impact": "no_impact",
        "cost_impact_value": "none",
        "custom_field_378362": "This idea was submitted via API and includes more fields in the request body to keep things spicy.",
        "due_date": (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d"),
        "private": False,
        "received_from_id": 8780450,
        "schedule_impact": "no_impact",
        "schedule_impact_value": "none",
        "skip_emails": True,
        "status":"Open",
        "title": "Complex Idea Submited by API"
    }
}
# create the item
tool_item = connection.tools.create_item(
    company_id=company["id"],
    project_id=project["id"],
    tool_id=tool["id"],
    data=data
)
# show created item
print(tool_item)

### `create_status()`
```python
"""
Create new status for a specific tool

Parameters
----------
company_id : int
    unique identifier for the company
tool_id : int
    unique identifier for the generic tool
data : dict
    request body data for the new item

Returns
-------
status_info : dict
    response from new status creation
"""
```

In [ ]:
# create new idea submission status
data = {
    "generic_tool_status": {
        "status_name": "In Review",
        "status": "Open"
    }
}
# create the item
tool_status = connection.tools.create_status(
    company_id=company["id"],
    tool_id=tool["id"],
    data=data
)
# show created item
print(tool_status)

### `delete_item()`
```python
"""
Updates item for a specific tool

Parameters
----------
company_id : int
    unique identifier for the company
project_id : int
    unique identifier for the project
tool_id : int
    unique identifier for the generic tool
item_id : int
    unique identifier for the item to change

Returns
-------
<status> : str
    status code and text of the delete
"""
```

In [ ]:
project = connection.projects.find(
    company_id=company["id"],
    identifier="Sandbox Test Project"
)
tool = connection.tools.find(
    company_id=company["id"],
    identifier="Idea Submission"
)
item = connection.tools.find_item(
    company_id=company["id"],
    project_id=project["id"],
    tool_id=tool["id"],
    identifier="Idea Submited by API 3"
)

# delete new idea submission item
_ = connection.tools.delete_item(
    company_id=company["id"],
    project_id=project["id"],
    tool_id=tool["id"],
    item_id=item["id"]
)

### `delete_status()`
```python
"""
Delete status for a specific tool

Parameters
----------
company_id : int
    unique identifier for the company
tool_id : int
    unique identifier for the generic tool
status_id : int
    status id for removal

Returns
-------
status_info : dict
    response from new status creation
"""
```

In [ ]:
project = connection.projects.find(
    company_id=company["id"],
    identifier="Sandbox Test Project"
)
tool = connection.tools.find(
    company_id=company["id"],
    identifier="Idea Submission"
)
item = connection.tools.find_item(
    company_id=company["id"],
    project_id=project["id"],
    tool_id=tool["id"],
    identifier="Idea Submited by API 3"
)

# delete new idea submission status
_ = connection.tools.delete_status(
    company_id=company["id"],
    tool_id=tool["id"],
    status_id=1234
)

### `find_item()`
```python
"""
Finds a specific generic tool item based on the identifier

Parameters
----------
company_id : int
    unique identifier for the company
project_id : int
    unique identifier for the project
tool_id : int
    unique identifier for the generic tool
identifier : int or str
    item id number or company name

Returns
-------
tool_item : dict
    response body for the given tool item
"""

In [13]:
# Find by Title
item1 = connection.tools.find_item(
    company_id=company["id"],
    project_id=project["id"],
    tool_id=tool["id"],
    identifier="Complex Idea Submited by API"
)

print(f'Found Idea: \'{item1["title"]}: {item1["id"]}\'')

AttributeError: 'Procore' object has no attribute '__tools__'

In [ ]:
#Find by ID
item2 = connection.tools.find_item(
    company_id=company["id"],
    project_id=project["id"],
    tool_id=tool["id"],
    identifier=5020162
)
print(item2["id"])
print(json.dumps(item2, indent=4))

In [ ]:
# find tool by title
tool_by_id = connection.tools.find(
    company_id=company["id"],
    identifier="Idea Submission"
)
print(f"{tool_by_id['id']}: {tool_by_id['title']}")

In [ ]:
# find tool by id (int)
tool_by_name = connection.tools.find(
    company_id=company["id"],
    identifier=287343
)
print(f"{tool_by_name['id']}: {tool_by_name['title']}")

In [ ]:
# no such tool
try:
    no_tool = connection.tools.find(
        company_id=company["id"],
        identifier="Fake Tool"
    )
except NotFoundItemError as e:
    print(e)

---